In [16]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
from time import sleep
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words

In [17]:
# pd option to show all columns
pd.set_option('display.max_columns', None)

In [27]:
url = "https://www.aliexpress.us/fn/search-pc/index"

payload = {
    "pageVersion": "76cea70e82a7e928f556677984719aa1",
    "target": "root",
    "data": {
        "g": "y",
        "SearchText": "usb power adapters",
        "sortType": "total_tranpro_desc",
        "origin": "y",
        "page": 1,
    },
    "eventName": "onChange",
    "dependency": []
}
headers = {
  'authority': 'www.aliexpress.us',
  'accept': '*/*',
  'accept-language': 'en-US,en;q=0.9',
  'bx-v': '2.5.5',
  'cache-control': 'no-cache',
  'content-type': 'application/json;charset=UTF-8',
  'cookie': 'acs_usuc_t=x_csrf=t_7qph5e082_&acs_rt=ad3b98d19c824028bfe738b80192e5bc; xman_t=/p2HW0NXX/G7elyMnxLztIuGtu00x3aEe4t8nEQRuKkhMFxFsrCM3kODCTDyVzL9; _ga_save=yes; xman_f=UJ2N2YLaQ58UcdcRnAB2wJvAQL8u2gi7CnAVvxQPu1u3uzRTwXzNXshoiL5ZmeTZRBx7kTeJspSEPOBkNmmTpKbxXFxI3AuMdGZm0lmLejxtnQctY7WC2Q==; intl_locale=en_US; aep_usuc_f=site=usa&c_tp=USD&region=US&b_locale=en_US; _m_h5_tk=f1ff74586b86758f7a2176e2db3d87cc_1701991093937; _m_h5_tk_enc=d353695bbd33e23e883de4d96cf7c7a5; xman_us_f=x_locale=en_US&x_l=0&x_c_chg=0&acs_rt=a39cce8f8a6f49b0814e0d694e9e746f; xlly_s=1; _gid=GA1.2.1256408879.1701988667; _gcl_au=1.1.242497071.1701988667; cna=Ojf5HRbkmWICAdilX5Zvsfqz; _fbp=fb.1.1701988673792.599238278; _ym_uid=1701988675955360031; _ym_d=1701988675; _ym_isad=1; _ym_visorc=b; AB_DATA_TRACK=472051_617391.455637_622998; AB_ALG=global_union_ab_exp_4%3D2; AB_STG=st_StrategyExp_1694492533501%23stg_687%2Cst_StrategyExp_1695711618618%23stg_685; _ga_VED1YSGNC7=GS1.1.1701988674.1.1.1701988709.25.0.0; _ga=GA1.1.c7fbc42c-1008-4ca9-934b-17ff58968ea3.1701988661135; cto_bundle=d-vGnl9taWwwYlZaM1gxSnVldW9NMlc5azdSRUlycEhvS1BiQVlxbUFtdGpLVGF2N1N1eklIcElSR3NZRE5vNnB5cFdPaXJUMkV2cm1iMyUyQiUyRkZOYnpvbmt1aG1qV2tkUTRMak1NSkJ5NDJyZVdqTFBvS0VpQ3VMTXA5T05DdmRielRzUDg1SXIyaG4zanpFb1VFSlRSd3JmQmE5WkNXREZwMk8ydW1qNnlJcjVjSWE0JTNE; g_state={"i_p":1701995966526,"i_l":1}; intl_common_forever=XBIyuBn1AUJJwqhs2jLBbljhFdpqAxs1zVjkJzzfXth9Sd47GzYDlg==; tfstk=e2BWJ1AgQ4058eGolzZV5kCCXWpQbawwvDtdjMHrvLp-J2I9uuvFaumdvM_sUYSp4eQfXE0P8L7EdeQFk_RR8_uIAGj3qkqkTeCfrGdyL9sndHId-uYUqSSlqpvL7PWQQgjuq3dZWR5qjLgMpPza38VjZtJdaH5l6MX078z6V0LIdt9lszZiz8HtBttjqEI5ByUFh6dkWULnpnWXVQTO2g5KQFTSqXiXsvtX7oZjtXDrdh_FZsXYuQKD0Vrbc4GnSvHEN9rj9XdJmnrTcogS1; l=fBrLuGoeP1kIprXhBOfaPurza77bGIR2muPzaNbMi9fP9A1erirhW1UqS0YwCneNesByP3S3hc2kB4L51yC2osbTKYnFdP1ZFdIvEn3A.; isg=BD8_xWRWZgTUAmIYiH8C84x-zhPJJJPGf4af_tEMsu414F1i0fBoFtK2J-i7jGs-',
  'origin': 'https://www.aliexpress.us',
  'pragma': 'no-cache',
  'referer': 'https://www.aliexpress.us/w/wholesale-lightsabers.html?g=y&SearchText=lightsabers&sortType=total_tranpro_desc',
  'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

In [28]:
product_list_dataframes = []

In [29]:
for page in range(1,21):
    print("Getting page: " + str(page))
    payload["data"]["page"] = page
    response = requests.request("POST", url, headers=headers, json=payload)
    r_data = response.json()
    product_list_df  = pd.json_normalize(r_data["data"]["result"]["mods"]["itemList"]["content"])
    product_list_dataframes.append(product_list_df)
    sleep(5)


Getting page: 1
Getting page: 2
Getting page: 3
Getting page: 4
Getting page: 5


In [30]:
product_list_df = pd.concat(product_list_dataframes)

In [31]:
product_list = product_list_df["productId"].tolist()
product_list

['3256805919085323',
 '3256802661722017',
 '3256805637165239',
 '3256805673493266',
 '3256804916918981',
 '3256803152352987',
 '3256805819848549',
 '3256805829584509',
 '2251832836254987',
 '3256804918132591',
 '3256805532452353',
 '3256805619056649',
 '3256804872085676',
 '3256805630651931',
 '3256805482722754',
 '3256805246469393',
 '3256805491367305',
 '3256805819875424',
 '3256805713915010',
 '3256804891418165',
 '3256805290776197',
 '3256803556008503',
 '3256805609837126',
 '3256804016949438',
 '3256805226729489',
 '3256802738668742',
 '3256805516566847',
 '3256805746211025',
 '3256805960431788',
 '3256805589755698',
 '3256804863487048',
 '3256805521066858',
 '3256805644512418',
 '3256805847225906',
 '3256805066704502',
 '3256805909858673',
 '3256803732911378',
 '3256805865807542',
 '3256805315623455',
 '3256804813149854',
 '3256803964983341',
 '3256805897698622',
 '3256806064686551',
 '2251832601971821',
 '3256802674534161',
 '3256803945230845',
 '3256805050937229',
 '32568054888

In [32]:
product_desc_data = []

In [33]:
product_counter = 1
for id in product_list:
    print(product_counter)
    url = f"https://www.aliexpress.us/item/{id}.html"
    print("Getting URL", url)
    payload = {}
    headers = {
        "authority": "www.aliexpress.us",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "en-US,en;q=0.9",
        "cache-control": "no-cache",
        "cookie": "acs_usuc_t=x_csrf=1cx6nzi0nrjga&acs_rt=7609757045ab48e3b9af6b52ac79ae7b; xman_t=cqkhR5z1OfevBKDeBU9757YJ4Fn8+bnpblet13TkBwu87SlQW7PGn+fglJlcRH0Y; xman_us_f=x_l=0&acs_rt=fe40ab160d894a48ba7d6a2ba0db0fcc; _ga=GA1.1.64002a25-b725-49e4-be7f-ad0dc422a247.1701991054061; _ga_save=yes; xman_f=m3OJpqJoIBIZSeDahoD5xGYePRe9A1NV84HtTfCgpEvoNLLed9n8A8k48cFpbR9nSYHpr3s4JPR3hl3lD1Njs20Zh60bdN7k7bdC2k88aVQDSHNh21BldQ==; aep_usuc_f=site=usa&c_tp=USD&region=US&b_locale=en_US; intl_common_forever=6nrbXpN0bCM04olN6B6GhZdmsMFq7Ol/Wt/1UETE8oq5yFAdUC8ZWw==; intl_locale=en_US; xman_us_f=x_locale=en_US&x_l=0&x_c_chg=0&acs_rt=a39cce8f8a6f49b0814e0d694e9e746f",
        "pragma": "no-cache",
        "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    soup = BeautifulSoup(response.text, 'html.parser')

    script_elem = soup.find_all('script')[0]
    script_data = script_elem.text.split("data: ")[~0].split("window.runParams")[0].strip()[:-1]
    desc_pattern = r'"descriptionUrl":"(.*?)"'

    desc_matches = re.findall(desc_pattern, script_data)

    des_match_url = desc_matches[0]

    response = requests.get(des_match_url, headers=headers, data=payload)

    desc_soup = BeautifulSoup(response.text, 'html.parser')

    desc_text = desc_soup.text.replace("\n", " ").replace("\t", " ").replace("\r", " ")

    metaData_pattern = r'"metaDataComponent":(.*?),"sellerComponent":'
    md_matches = re.findall(metaData_pattern, script_data)
    meta_data_str = md_matches[0]
    meta_data = json.loads(meta_data_str)

    meta_data.update({"id": id, "desc": desc_text})
    # temp = pd.DataFrame(meta_data)
    # print(temp)
    # print(meta_data)
    product_desc_data.append(meta_data)
    product_counter += 1
    sleep(5)

1
Getting URL https://www.aliexpress.us/item/3256805919085323.html
2
Getting URL https://www.aliexpress.us/item/3256802661722017.html
3
Getting URL https://www.aliexpress.us/item/3256805637165239.html
4
Getting URL https://www.aliexpress.us/item/3256805673493266.html
5
Getting URL https://www.aliexpress.us/item/3256804916918981.html
6
Getting URL https://www.aliexpress.us/item/3256803152352987.html
7
Getting URL https://www.aliexpress.us/item/3256805819848549.html
8
Getting URL https://www.aliexpress.us/item/3256805829584509.html
9
Getting URL https://www.aliexpress.us/item/2251832836254987.html
10
Getting URL https://www.aliexpress.us/item/3256804918132591.html
11
Getting URL https://www.aliexpress.us/item/3256805532452353.html
12
Getting URL https://www.aliexpress.us/item/3256805619056649.html
13
Getting URL https://www.aliexpress.us/item/3256804872085676.html
14
Getting URL https://www.aliexpress.us/item/3256805630651931.html
15
Getting URL https://www.aliexpress.us/item/32568054827

In [34]:
product_desc_data

[{'keywords': 'Aliexpress, Aliexpress.com, Online shopping, Automotive, Phones & Accessories, Computers & Electronics, Fashion, Beauty & Health, Home & Garden, Toys & Sports, Weddings & Events',
  'canonicalTag': '<link rel="canonical" href="https://www.aliexpress.com/item/1005006105400075.html">',
  'hreflangTag': '<link rel="alternate" hreflang="en" href="https://www.aliexpress.com/item/1005006105400075.html"/>\n<link rel="alternate" hreflang="id" href="https://id.aliexpress.com/item/1005006105400075.html"/>\n<link rel="alternate" hreflang="ko" href="https://ko.aliexpress.com/item/1005006105400075.html"/>\n<link rel="alternate" hreflang="ar" href="https://ar.aliexpress.com/item/1005006105400075.html"/>\n<link rel="alternate" hreflang="de" href="https://de.aliexpress.com/item/1005006105400075.html"/>\n<link rel="alternate" hreflang="es" href="https://es.aliexpress.com/item/1005006105400075.html"/>\n<link rel="alternate" hreflang="fr" href="https://fr.aliexpress.com/item/10050061054000

In [35]:
closed_class_stop_words = ['a','the','an','and','or','but','about','above','after','along','amid','among', \
                           'as','at','by','for','from','in','into','like','minus','near','of','off','on', \
                           'onto','out','over','past','per','plus','since','till','to','under','until','up', \
                           'via','vs','with','that','can','cannot','could','may','might','must', \
                           'need','ought','shall','should','will','would','have','had','has','having','be', \
                           'is','am','are','was','were','being','been','get','gets','got','gotten', \
                           'getting','seem','seeming','seems','seemed', \
                           'enough', 'both', 'all', 'your' 'those', 'this', 'these', \
                           'their', 'the', 'that', 'some', 'our', 'no', 'neither', 'my', \
                           'its', 'his' 'her', 'every', 'either', 'each', 'any', 'another', \
                           'an', 'a', 'just', 'mere', 'such', 'merely' 'right', 'no', 'not', \
                           'only', 'sheer', 'even', 'especially', 'namely', 'as', 'more', \
                           'most', 'less' 'least', 'so', 'enough', 'too', 'pretty', 'quite', \
                           'rather', 'somewhat', 'sufficiently' 'same', 'different', 'such', \
                           'when', 'why', 'where', 'how', 'what', 'who', 'whom', 'which', \
                           'whether', 'why', 'whose', 'if', 'anybody', 'anyone', 'anyplace', \
                           'anything', 'anytime' 'anywhere', 'everybody', 'everyday', \
                           'everyone', 'everyplace', 'everything' 'everywhere', 'whatever', \
                           'whenever', 'whereever', 'whichever', 'whoever', 'whomever' 'he', \
                           'him', 'his', 'her', 'she', 'it', 'they', 'them', 'its', 'their','theirs', \
                           'you','your','yours','me','my','mine','I','we','us','much','and/or'
                           ]

In [36]:
cur_prod_listings = pd.DataFrame(product_desc_data)
f = open("poweradapters_non_DS.txt", 'w', encoding = "utf-8")
fields_to_access = ["keywords", "ogTitle", "description", "ogDescription", "title", "desc"]
reducer = PorterStemmer()
for index, row in cur_prod_listings.iterrows():
    for l in fields_to_access: #replaces nums w/ words and gets rid of special chars
        unmodified_field = ""
        # print(row[l])
        for s in row[l]:
            if s.isalpha() or s.isspace():
                unmodified_field += s
            elif s in "1234567890":
                unmodified_field += num2words(int(s)) + " "
        # print(unmodified_field)
       #removes special chars and punctuation
        tokenized_field = word_tokenize(unmodified_field) #tokenizes word
        modified_field = [reducer.stem(w) for w in tokenized_field if w not in closed_class_stop_words] #remove stop words
        # print(unmodified_field)
        for word in modified_field:
            print(word + " ")
            f.write(word + " ")
    f.write("\n")

onlin 
shop 
automot 
phone 
accessori 
comput 
electron 
fashion 
beauti 
health 
home 
garden 
toy 
sport 
wed 
event 
one 
five 
five 
four 
us 
seven 
three 
offnin 
eight 
zero 
zero 
zero 
pa 
car 
vacuum 
cleaner 
mini 
clean 
machin 
strong 
suction 
usb 
handheld 
car 
home 
applianc 
portabl 
wireless 
cleaner 
onlin 
shop 
cheapest 
price 
automot 
phone 
accessori 
comput 
electron 
fashion 
beauti 
health 
home 
garden 
toy 
sport 
wed 
event 
els 
enjoy 
free 
ship 
worldwid 
limit 
time 
sale 
easi 
return 
smarter 
shop 
better 
live 
nine 
eight 
zero 
zero 
zero 
pa 
car 
vacuum 
cleaner 
mini 
clean 
machin 
strong 
suction 
usb 
handheld 
car 
home 
applianc 
portabl 
wireless 
cleaner 
introductionproduct 
name 
handheld 
vehicl 
vacuum 
cleanerproduct 
functionsmainli 
clean 
car 
vacuum 
blow 
dust 
blow 
balloon 
onvers 
introductionupd 
versionproduct 
materi 
absmotor 
new 
upgrat 
brushless 
motorvacuum 
degre 
nine 
eight 
zero 
zero 
zero 
parat 
speed 
thr

In [72]:
cur_prod_listings.loc[0].at("")

,keywords,canonicalTag,hreflangTag,ogTitle,description,mediaTag,ogurl,amphtmlTag,ogDescription,title,id,desc
0,led strip lights rgb app control color changin...,"<link rel=""canonical"" href=""https://www.aliexp...","<link rel=""alternate"" hreflang=""en"" href=""http...",0.98US $ 93% OFF|Led Strip Lights Rgb Control ...,Shop led strip lights with fast shipping and f...,"<link rel=""alternate"" media=""only screen and (...",//www.aliexpress.com/item/3256805388409831.html,<meta>,"Smarter Shopping, Better Living! Aliexpress.com",Led Strip Lights Rgb Control Color Changing 24...,3256805388409831,"How does it work?In the first step, you need t..."
1,"led strip lights, LED Strip, LED Lighting, Lig...","<link rel=""canonical"" href=""https://www.aliexp...","<link rel=""alternate"" hreflang=""en"" href=""http...","0.99US $ 94% OFF|Led Strip Lights Rgb 5050 ,5v...",Buy led strip lights with free shipping and fr...,"<link rel=""alternate"" media=""only screen and (...",//www.aliexpress.com/item/3256805299570315.html,<meta>,"Smarter Shopping, Better Living! Aliexpress.com","Led Strip Lights Rgb 5050 ,5v 1m-30m,16 Millio...",3256805299570315,Upgraded Brighter LED chips and durable copper...


Test Area - ignore

In [ ]:
id = 3256805027484090

In [ ]:
product_desc_data

In [ ]:
s_text = soup.text.replace("\n", " ").replace("\t", " ").replace("\r", " ")
' '.join(s_text.split())